In [1]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
% cd /content/drive/MyDrive/Colab Notebooks/xlm-roberta/ICHCL_baseline

/content/drive/MyDrive/Colab Notebooks/xlm-roberta/ICHCL_baseline


In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import re
import json

import time
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

import torch
import tensorflow as tf

## **Load test and train**

In [ ]:
% cd /content/drive/MyDrive/Colab Notebooks/xlm-roberta

/content/drive/MyDrive/Colab Notebooks/xlm-roberta


In [ ]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [ ]:
X_train=df_train['features']
X_val=df_test['features']
y_train=df_train['label']
y_val = df_test['label']

In [ ]:
X = np.concatenate((X_train, X_val))
y = np.concatenate((y_train, y_val))

## **Load model**

In [ ]:
! pip install -q transformers

     |████████████████████████████████| 2.6 MB 8.0 MB/s 
     |████████████████████████████████| 636 kB 53.6 MB/s 
     |████████████████████████████████| 895 kB 61.5 MB/s 
     |████████████████████████████████| 3.3 MB 31.8 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
  
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
from sklearn.decomposition import PCA

## **Embedding the traning data**

In [ ]:
% cd /content/drive/MyDrive/Colab Notebooks/xlm-roberta/X

/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X


In [ ]:
print(len(X))

5740


In [ ]:
X=X.tolist()

In [ ]:
tokenized_input = tokenizer(
 X,
 padding=True,
 truncation=False,
 return_tensors='pt'
)


In [ ]:
tokenized_input

{'input_ids': tensor([[     0, 104257,    287,  ...,      1,      1,      1],
        [     0,  70827,   1029,  ...,      1,      1,      1],
        [     0,   3835,  78431,  ...,      1,      1,      1],
        ...,
        [     0, 122585, 171495,  ...,      1,      1,      1],
        [     0, 104257,    287,  ...,      1,      1,      1],
        [     0,  10399,  41019,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
tokenized_input['input_ids'] = tokenized_input['input_ids'].cpu().detach().numpy()
tokenized_input['attention_mask'] = tokenized_input['attention_mask'].cpu().detach().numpy()

In [ ]:
tokenized_input['input_ids'].shape

(5740, 184)

In [ ]:
output = []

j = 1

path="/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output"
pca=PCA(n_components=30)

for i in range(len(X)):
    input = {}
    input['input_ids'] = torch.from_numpy(tokenized_input['input_ids'][i].reshape(1, 184))
    input['attention_mask'] = torch.from_numpy(tokenized_input['attention_mask'][i].reshape(1, 184))

    model_output = model(**input)
    model_output = model_output.logits.cpu().detach().numpy()

    model_output=model_output.reshape(184, 250002)
    model_output=pca.fit_transform(model_output)

    output.append(model_output)

    if ((i + 1) % 20) == 0:
        file_name = path + str(j) + ".txt"
        with open(file_name, "wb") as fp:   #Pickling
            pickle.dump(output, fp)
        print(file_name + " done")
        output = []
        j += 1
        
file_name = path + str(j) + ".txt"
with open(file_name, "wb") as fp:   #Pickling
    pickle.dump(output, fp)
print(file_name + " done")
output = []

/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output1.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output2.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output3.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output4.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output5.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output6.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output7.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output8.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output9.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output10.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output11.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output12.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output13.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output14.txt done
/content/drive/

In [ ]:
output = []
path="/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output"
for i in range(288):
    file_name = path + str(i + 1) + ".txt"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output1.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output2.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output3.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output4.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output5.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output6.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output7.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output8.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output9.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output10.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output11.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output12.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output13.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X/output14.txt done
/content/drive/

In [ ]:
X = output
output = []

In [ ]:
print(len(X))

5740


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train=np.array(X_train)
y_train=np.array(y_train)

In [ ]:
X_val=np.array(X_val)

In [ ]:
X_val.shape

(1148, 184, 30)

In [ ]:
X_train=X_train.reshape(4592,5520)

In [ ]:
X_train.shape
#y_train.shape

(4592, 5520)

In [ ]:
X_val=X_val.reshape(1148,5520)

## **Logistic regression**

In [ ]:
# Loading the Model

classifier = LogisticRegression(max_iter=500)

In [ ]:
# Training the Model

classifier.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# Evaluating the Model

y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.51      0.55      0.53       547
        NONE       0.56      0.52      0.54       601

    accuracy                           0.54      1148
   macro avg       0.54      0.54      0.54      1148
weighted avg       0.54      0.54      0.54      1148



## **Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.50      0.04      0.07       547
        NONE       0.52      0.97      0.68       601

    accuracy                           0.52      1148
   macro avg       0.51      0.50      0.37      1148
weighted avg       0.51      0.52      0.39      1148



##**Stochastic Gradient Descent**

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
classifier = SGDClassifier()
classifier.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.51      0.49      0.50       547
        NONE       0.55      0.57      0.56       601

    accuracy                           0.53      1148
   macro avg       0.53      0.53      0.53      1148
weighted avg       0.53      0.53      0.53      1148



## **K Nearest Neighbours**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
classifier = KNeighborsClassifier()
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.62      0.51      0.56       547
        NONE       0.62      0.72      0.66       601

    accuracy                           0.62      1148
   macro avg       0.62      0.61      0.61      1148
weighted avg       0.62      0.62      0.61      1148



## **Decision Trees**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.50      0.53      0.51       547
        NONE       0.55      0.52      0.53       601

    accuracy                           0.52      1148
   macro avg       0.52      0.52      0.52      1148
weighted avg       0.52      0.52      0.52      1148



## **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.59      0.63      0.61       547
        NONE       0.64      0.60      0.62       601

    accuracy                           0.61      1148
   macro avg       0.61      0.61      0.61      1148
weighted avg       0.62      0.61      0.61      1148



## **Support Vector Machine**

In [ ]:
from sklearn.svm import SVC

In [ ]:
classifier = SVC()
classifier.fit(X_train, y_train)

ValueError: ignored

In [ ]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

## **Ensemble - Voting**

In [ ]:
# Logistic Regression

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)

## **NN**

In [ ]:
le = LabelEncoder() #label encoding labels for training Dense Neural Network
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

In [ ]:
model = Sequential(
    [
        Dense(128, activation="tanh"),
        Dense(128, activation="tanh"),
        Dense(64, activation="tanh"),
        Dense(64, activation="tanh"),
        Dropout(0.1),
        Dense(32, activation="tanh"),
        Dropout(0.1),
        Dense(1, activation="sigmoid"),
    ]
)
model.compile('adam', loss='binary_crossentropy', metrics = ['accuracy']) #compiling a neural network with 3 layers for classification

In [ ]:
model.fit(X_train, y_train, epochs = 20, batch_size = 32)

Epoch 1/20
144/144 [==============================] - 1s 5ms/step - loss: 0.6781 - accuracy: 0.5640
Epoch 2/20
144/144 [==============================] - 0s 3ms/step - loss: 0.6804 - accuracy: 0.5623
Epoch 3/20
144/144 [==============================] - 1s 3ms/step - loss: 0.6748 - accuracy: 0.5688
Epoch 4/20
144/144 [==============================] - 0s 3ms/step - loss: 0.6784 - accuracy: 0.5773
Epoch 5/20
144/144 [==============================] - 1s 4ms/step - loss: 0.6751 - accuracy: 0.5799
Epoch 6/20
144/144 [==============================] - 0s 3ms/step - loss: 0.6758 - accuracy: 0.5747
Epoch 7/20
144/144 [==============================] - 0s 3ms/step - loss: 0.6750 - accuracy: 0.5812
Epoch 8/20
144/144 [==============================] - 0s 3ms/step - loss: 0.6695 - accuracy: 0.5886
Epoch 9/20
144/144 [==============================] - 0s 3ms/step - loss: 0.6701 - accuracy: 0.5871
Epoch 10/20
144/144 [==============================] - 0s 3ms/step - loss: 0.6690 - accuracy: 0.5862

In [ ]:
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype('int64')
y_pred = y_pred.reshape(len(y_pred))    

In [ ]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.75      0.61       547
           1       0.61      0.36      0.45       601

    accuracy                           0.54      1148
   macro avg       0.56      0.55      0.53      1148
weighted avg       0.57      0.54      0.53      1148



## **Training on Entire Data**

In [ ]:
X = output
output = []

In [ ]:
print(len(X))

5740


In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
y.shape

(5740,)

In [ ]:
X = X.reshape(5740,5520)

In [ ]:
# Logistic Regression

lr = LogisticRegression(max_iter=500)
lr.fit(X, y)
lr_pred = lr.predict(X)
print(classification_report(y, lr_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

         HOF       0.87      0.88      0.88      2841
        NONE       0.89      0.88      0.88      2899

    accuracy                           0.88      5740
   macro avg       0.88      0.88      0.88      5740
weighted avg       0.88      0.88      0.88      5740



In [ ]:
# Naive Bayes

from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X, y)
nb_pred = nb.predict(X)
print(classification_report(y, nb_pred))

              precision    recall  f1-score   support

         HOF       0.62      0.04      0.07      2841
        NONE       0.51      0.98      0.67      2899

    accuracy                           0.51      5740
   macro avg       0.56      0.51      0.37      5740
weighted avg       0.56      0.51      0.37      5740



In [ ]:
# Stochastic Gradient Descent

from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd.fit(X, y)
sgd_pred = sgd.predict(X)
print(classification_report(y, sgd_pred))

              precision    recall  f1-score   support

         HOF       0.80      0.63      0.71      2841
        NONE       0.70      0.85      0.77      2899

    accuracy                           0.74      5740
   macro avg       0.75      0.74      0.74      5740
weighted avg       0.75      0.74      0.74      5740



In [ ]:
# K Nearest Neighbours

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X, y)
knn_pred = knn.predict(X)
print(classification_report(y, knn_pred))

              precision    recall  f1-score   support

         HOF       0.78      0.69      0.73      2841
        NONE       0.73      0.81      0.77      2899

    accuracy                           0.75      5740
   macro avg       0.75      0.75      0.75      5740
weighted avg       0.75      0.75      0.75      5740



In [ ]:
# Decision Trees

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X, y)
dt_pred = dt.predict(X)
print(classification_report(y, dt_pred))

              precision    recall  f1-score   support

         HOF       1.00      1.00      1.00      2841
        NONE       1.00      1.00      1.00      2899

    accuracy                           1.00      5740
   macro avg       1.00      1.00      1.00      5740
weighted avg       1.00      1.00      1.00      5740



In [ ]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X, y)
rf_pred = rf.predict(X)
print(classification_report(y, rf_pred))

              precision    recall  f1-score   support

         HOF       1.00      1.00      1.00      2841
        NONE       1.00      1.00      1.00      2899

    accuracy                           1.00      5740
   macro avg       1.00      1.00      1.00      5740
weighted avg       1.00      1.00      1.00      5740



In [ ]:
# Support Vector Machine

from sklearn.svm import SVC
svc = SVC()
svc.fit(X, y)
svc_pred = svc.predict(X)
print(classification_report(y, svc_pred))

              precision    recall  f1-score   support

         HOF       0.84      0.81      0.83      2841
        NONE       0.82      0.85      0.84      2899

    accuracy                           0.83      5740
   macro avg       0.83      0.83      0.83      5740
weighted avg       0.83      0.83      0.83      5740



## **Load Testing Data**

In [ ]:
% cd /content/drive/MyDrive/Colab Notebooks/xlm-roberta/ICHCL_baseline

/content/drive/MyDrive/Colab Notebooks/xlm-roberta/ICHCL_baseline


In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import re
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout


from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import stemmer as hindi_stemmer

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
english_stopwords = stopwords.words("english")
with open('final_stopwords.txt', encoding = 'utf-8') as f:
    hindi_stopwords = f.readlines()
    for i in range(len(hindi_stopwords)):
        hindi_stopwords[i] = re.sub('\n','',hindi_stopwords[i])
stopwords = english_stopwords + hindi_stopwords
english_stemmer = SnowballStemmer("english")

In [ ]:
def te_flatten(d):
    flat_text = []
    flat_text.append({
        'tweet_id':d['tweet_id'],
        'text':d['tweet'],
    })

    for i in d['comments']:
            flat_text.append({
                'tweet_id':i['tweet_id'],
                'text':flat_text[0]['text'] + i['tweet'],
            })
            if 'replies' in i.keys():
                for j in i['replies']:
                    flat_text.append({
                        'tweet_id':j['tweet_id'],
                        'text':flat_text[0]['text'] + i['tweet'] + j['tweet'],
                    })
    return flat_text

In [ ]:
regex_for_english_hindi_emojis="[^a-zA-Z#\U0001F300-\U0001F5FF'|'\U0001F600-\U0001F64F'|'\U0001F680-\U0001F6FF'|'\u2600-\u26FF\u2700-\u27BF\u0900-\u097F]"
def clean_tweet(tweet):
    tweet = re.sub(r"@[A-Za-z0-9]+",' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet)
    tweet = re.sub(regex_for_english_hindi_emojis,' ', tweet)
    tweet = re.sub("RT ", " ", tweet)
    tweet = re.sub("\n", " ", tweet)
    tweet = re.sub(r" +", " ", tweet)
    tokens = []
    for token in tweet.split():
        if token not in stopwords:
            token = english_stemmer.stem(token)
            token = hindi_stemmer.hi_stem(token)
            tokens.append(token)
    return " ".join(tokens)

In [ ]:
test_directories = []
for i in glob("test/*/"):
    for j in glob(i+'*/'):
        test_directories.append(j)

In [ ]:
test_directories

['test/covid crisis/1396844054818680835/',
 'test/covid crisis/1395277604782022659/',
 'test/covid crisis/1385891002209038343/',
 'test/covid crisis/1398274810652753924/',
 'test/religious controversies/1393528907102515200/',
 'test/religious controversies/1392808101007478794/',
 'test/indian politics/1395657670762405888/',
 'test/indian politics/1396686927227588609/',
 'test/indian politics/1390362987995623428/',
 'test/israel/1392157687723347968/',
 'test/israel/1392413634773409792/',
 'test/wuhan virus/1392426204775391233/',
 'test/bantwitter/1397223849830359052/',
 'test/bantwitter/1397136107108663298/',
 'test/casteism/1395755293213003779/',
 'test/casteism/1394882097559400449/']

In [ ]:
data = []
for i in test_directories:
    with open(i+'data.json', encoding='utf-8') as f:
        data.append(json.load(f))

In [ ]:
test_tweetid_data = []
#for test
for i in range(len(data)):
    for j in te_flatten(data[i]):
        test_tweetid_data.append(j)

In [ ]:
test_df = pd.DataFrame(test_tweetid_data, columns = test_tweetid_data[0].keys(), index = None)

In [ ]:
test_df.head()

,tweet_id,text
0,1396844054818680835,Bhadva Ramdev was brought to a debate about Al...
1,1396844158283776004,Bhadva Ramdev was brought to a debate about Al...
2,1397043581446098945,Bhadva Ramdev was brought to a debate about Al...
3,1398265913749635073,Bhadva Ramdev was brought to a debate about Al...
4,1396852220268716032,Bhadva Ramdev was brought to a debate about Al...


In [ ]:
test_tweets = test_df.text
tweet_ids = test_df.tweet_id

In [ ]:
X_test = [clean_tweet(tweet) for tweet in test_tweets]

## **Load Model for Embedding Test Data**

In [ ]:
# Load Model

! pip install -q transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

from sklearn.decomposition import PCA

     |████████████████████████████████| 2.6 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 54.9 MB/s 
     |████████████████████████████████| 3.3 MB 53.5 MB/s 
     |████████████████████████████████| 636 kB 72.0 MB/s 


Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

## **Embedding Test Data**

In [ ]:
% cd /content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test

/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test


In [ ]:
tokenized_input = tokenizer(
 X_test,
 padding=True,
 truncation=False,
 return_tensors='pt'
)

In [ ]:
tokenized_input['input_ids'] = tokenized_input['input_ids'].cpu().detach().numpy()
tokenized_input['attention_mask'] = tokenized_input['attention_mask'].cpu().detach().numpy()

In [ ]:
tokenized_input['input_ids'].shape

(1348, 159)

In [ ]:
output = []

j = 1

path="/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output"
pca=PCA(n_components=30)

for i in range(len(X_test)):
    input = {}
    input['input_ids'] = torch.from_numpy(tokenized_input['input_ids'][i].reshape(1, 159))
    input['attention_mask'] = torch.from_numpy(tokenized_input['attention_mask'][i].reshape(1, 159))

    model_output = model(**input)
    model_output = model_output.logits.cpu().detach().numpy()

    model_output=model_output.reshape(159, 250002)
    model_output=pca.fit_transform(model_output)

    output.append(model_output)

    if ((i + 1) % 20) == 0:
        file_name = path + str(j) + ".txt"
        with open(file_name, "wb") as fp:   #Pickling
            pickle.dump(output, fp)
        print(file_name + " done")
        output = []
        j += 1
        
file_name = path + str(j) + ".txt"
with open(file_name, "wb") as fp:   #Pickling
    pickle.dump(output, fp)
print(file_name + " done")
output = []

/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output1.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output2.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output3.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output4.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output5.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output6.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output7.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output8.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output9.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output10.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output11.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output12.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output13.txt done
/content/drive/MyDriv

In [ ]:
output = []
path="/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output"
for i in range(68):
    file_name = path + str(i + 1) + ".txt"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output1.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output2.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output3.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output4.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output5.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output6.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output7.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output8.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output9.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output10.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output11.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output12.txt done
/content/drive/MyDrive/Colab Notebooks/xlm-roberta/X_test/output13.txt done
/content/drive/MyDriv

In [ ]:
X_test = output
output = []

In [ ]:
print(len(X_test))

1348


In [ ]:
X_test=np.array(X_test)

In [ ]:
X_test.shape

(1348, 159, 30)

In [ ]:
X_test = X_test.reshape(1348, 4770)

In [ ]:
X_test = X_test.tolist()
zero = [0] * 750
for i in range(len(X_test)):
  X_test[i] = X_test[i] + zero
X_test = np.array(X_test)
X_test.shape
X_test = X_test.reshape(1348, 5520)

In [ ]:
y_test = svc.predict(X_test)

In [ ]:
test_df.head()

,tweet_id,text,label
0,1396844054818680835,Bhadva Ramdev was brought to a debate about Al...,HOF
1,1396844158283776004,Bhadva Ramdev was brought to a debate about Al...,HOF
2,1397043581446098945,Bhadva Ramdev was brought to a debate about Al...,HOF
3,1398265913749635073,Bhadva Ramdev was brought to a debate about Al...,HOF
4,1396852220268716032,Bhadva Ramdev was brought to a debate about Al...,HOF


In [ ]:
len(y_test)

1348

In [ ]:
y_test = y_test.tolist()

In [ ]:
y_test

['HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',


In [ ]:
y_test

['HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',


In [ ]:
test_df['label'] = y_test

In [ ]:
test_df.head()

,tweet_id,text,label
0,1396844054818680835,Bhadva Ramdev was brought to a debate about Al...,HOF
1,1396844158283776004,Bhadva Ramdev was brought to a debate about Al...,HOF
2,1397043581446098945,Bhadva Ramdev was brought to a debate about Al...,HOF
3,1398265913749635073,Bhadva Ramdev was brought to a debate about Al...,HOF
4,1396852220268716032,Bhadva Ramdev was brought to a debate about Al...,HOF


In [ ]:
tweet_id = test_df['tweet_id']

In [ ]:
tweet_id = np.array(tweet_id)

In [ ]:
submission = {'tweet_id': tweet_id, 'label':y_test}
submission = pd.DataFrame(submission)

In [ ]:
submission.head()

,tweet_id,label
0,1396844054818680835,HOF
1,1396844158283776004,HOF
2,1397043581446098945,HOF
3,1398265913749635073,HOF
4,1396852220268716032,HOF


In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/xlm_roberta_svm_submission.csv', index = False)

In [ ]:
! ls

output10.txt  output23.txt  output36.txt  output49.txt	output61.txt
output11.txt  output24.txt  output37.txt  output4.txt	output62.txt
output12.txt  output25.txt  output38.txt  output50.txt	output63.txt
output13.txt  output26.txt  output39.txt  output51.txt	output64.txt
output14.txt  output27.txt  output3.txt   output52.txt	output65.txt
output15.txt  output28.txt  output40.txt  output53.txt	output66.txt
output16.txt  output29.txt  output41.txt  output54.txt	output67.txt
output17.txt  output2.txt   output42.txt  output55.txt	output68.txt
output18.txt  output30.txt  output43.txt  output56.txt	output6.txt
output19.txt  output31.txt  output44.txt  output57.txt	output7.txt
output1.txt   output32.txt  output45.txt  output58.txt	output8.txt
output20.txt  output33.txt  output46.txt  output59.txt	output9.txt
output21.txt  output34.txt  output47.txt  output5.txt
output22.txt  output35.txt  output48.txt  output60.txt
